# Getting started

TypeScript函数式编程基础

## 类型类(Type Class)与实例(instance)

### Type Class
> 在函数编程中，开发者常常会去定义通过定义一组“操作”(函数/变量)，并将这些“操作”应用于某个类型。Type Class强调的是类型行为的抽象和扩展。
在TypeScript中，常用 `interface` 声明一个Type Class，例如：

In [1]:
interface Eq<A> {
  /** returns `true` if `x` is equal to `y` */
  readonly equals: (x: A, y: A) => boolean
}

上面示例中，类型 `A` 属于类型类 `Eq`，并为类型 `A` 应用 `equals` 函数。

### instance
instance即为某个具体类型的类型类实现，必须满足以下定律：
* 自反性（Reflexivity）：对于集合A中的任意x，equals(x, x) === true
* 对称性（Symmetry）：对于集合A中的任意x、y，equals(x, y) === equals(y, x)
* 传递性（Transitivity）：对于集合A中的任意x、y、z，如果equals(x, y) === true 且 equals(y, z) === true，则 equals(x, z) === true

In [2]:
const eqNumber: Eq<number> = {
  equals: (x, y) => x === y
}

[维基百科](https://zh.wikipedia.org/wiki/%E7%B1%BB%E5%9E%8B%E7%B1%BB)中对类型类的解释中有提到，一个类型类往往会涉及到一个**类型类T**和**类型变量a**，我们可以据此理解上述示例中:
```
                ┌─── 类型类
                |    ┌─── 类型变量number
                ▼    ▼
const eqNumber: Eq<number>
```

### Type class 示例 - `Ord`

上面的案例中，使用类型类`Eq<A>`可以实现判断两个元素是否相等，借助于`Eq<A>`，可以实现类型元素的排序。

In [ ]:
/**
 * 排序结果类型，-1表示小于，0表示等于，1表示大于
 */
type Ordering = -1 | 0 | 1;

/**
 * Ord 类型类，扩展自 Eq<A>
 * compare: 比较两个元素，返回 Ordering 类型
 */
interface Ord<A> extends Eq<A> {
  readonly compare: (x: A, y: A) => Ordering;
}

/**
 * 默认的相等判断函数：先用===判断，如果不等则用compare判断是否为0
 * compare为Ord类型的比较函数
 */
const equalsDefault =
  <A>(compare: Ord<A>['compare']): Eq<A>['equals'] =>
  (first, second) => first === second || compare(first, second) === 0;

/**
 * 根据compare函数快速构造Ord实例
 * 自动生成equals和compare方法
 */
const fromCompare =
  <A>(compare: Ord<A>['compare']): Ord<A> => ({
    equals: equalsDefault(compare),
    compare
  })

/**
 * number类型的Ord实例，按数值大小比较
 */
const ordNumber: Ord<number> = fromCompare((x, y) => x < y ? -1 : x > y ? 1 : 0);

// ordNumber.compare示例
console.log(ordNumber.compare(1, 2)) // -1
console.log(ordNumber.compare(1, 1)) // 0
console.log(ordNumber.compare(2, 1)) // 1

/**
 * User类型定义
 */
type User = {
  name: string
  age: number
}

/**
 * 按年龄比较User的Ord实例
 */
const ordUserByAge: Ord<User> = fromCompare((x, y) => ordNumber.compare(x.age, y.age));

console.log(ordUserByAge.compare({ name: 'Alice', age: 30 }, { name: 'Bob', age: 25 })) // 1

/**
 * 使用Ord，可以实现最小值、最大值比较
 */
const min = <A>(O: Ord<A>) => (x: A, y: A) => O.compare(x, y) === 1 ? y : x;
const reverse = <A>(O: Ord<A>) => fromCompare((x: A, y: A) => O.compare(y, x))
const max = <A>(O: Ord<A>) => (x: A, y: A) => min(reverse(O))(x, y)

console.log(min(ordNumber)(3, 2)) // 2
console.log(max(ordNumber)(3, 2)) // 3
console.log(min(ordUserByAge)({ name: 'Alice', age: 30 }, { name: 'Bob', age: 25 })) // { name: 'Bob', age: 25 }
console.log(max(ordUserByAge)({ name: 'Alice', age: 30 }, { name: 'Bob', age: 25 })) // { name: 'Alice', age: 30 }

## Semigroup

在函数式编程中，**Semigroup**（半群）是一种非常基础的类型类。它描述了一类可以“合并”自身元素的类型，Semigroup可以用以下形式描述
- (A, *)
其中`A`为一个非空集合，`*`为`A`上的一个二元结合运算（通常称之为`concat`或者`combine`），也即函数。它接受`A`的两个元素作为输入，并返回`A`的一个元素作为输出：
> *: (x: A, y: A) => A

Semigroup的几个典型例子:
* (number, *): (x: number, y: number) => x * y
* (string, *): (x: string, y: string) => x + y
* (boolean, &&): (x: boolean, y: boolean) => x && y
结合律: </br>
Semigroup定义中，要求对元素的操作满足**结合律**：对于任意a、b、c：`concat(a, concat(b, c)) === (concat(a, b), c)`


### Semigroup type class definition

In [3]:
interface Semigroup<A> {
  readonly concat: (x: A, y: A) => A;
}

/**
 * A number semigroup under addition
 */
const semigroupSum: Semigroup<number> = {
  concat: (x, y) => x + y
}

/**
 * A string semigroup under `concat`
 */
const semigroupString: Semigroup<string> = {
  concat: (x, y) => x + y
}

/**
 * Always return the first argument
 */
function getFirstSemigroup<A=never>(): Semigroup<A> {
  return {
    concat: (x, y) => x
  }
}

/**
 * Always return the second argument
 */
function getLastSemigroup<A=never>(): Semigroup<A> {
  return {
    concat: (x, y) => y
  }
}